In [109]:
import polars as pl

In [110]:
from zipfile import ZipFile
import glob as glob

In [118]:
glob.glob('data/*')

['data\\lahman_1871-2024_csv', 'data\\league_era.csv', 'data\\league_slg.csv']

In [117]:
(data := pl.read_csv("./data/lahman_1871-2024_csv/Pitching.csv",
                           columns = ['playerID', 'yearID', 'ER', 'lgID', 'IPouts']
                          )
)
league_era = (
    data
    .filter(pl.col('yearID') >= 1946)
    .group_by('yearID', 'lgID')
    .agg(
        tot_er_allowed = pl.col('ER').sum(),
        tot_innings_pitched = pl.col('IPouts').sum(),
    )
    .with_columns( league_wide_ERA = (pl.col('tot_er_allowed')/pl.col('tot_innings_pitched'))*27)
    .with_columns( DH_status = pl.when(  ((pl.col('lgID') == 'AL') & (pl.col('yearID') >= 1973)) |
                                         ((pl.col('lgID') == 'NL') & (pl.col('yearID') >= 2022))
                                      )
                   .then(pl.lit('DH'))
                   .otherwise(pl.lit('No DH'))
                 )
)

league_era.select(
    pl.col('yearID','lgID','league_wide_ERA','DH_status')).head(10)

yearID,lgID,league_wide_ERA,DH_status
i64,str,f64,str
2000,"""AL""",4.921603,"""DH"""
2024,"""NL""",4.138092,"""DH"""
1982,"""NL""",3.60855,"""No DH"""
2023,"""AL""",4.280259,"""DH"""
2014,"""AL""",3.819454,"""DH"""
1964,"""NL""",3.535345,"""No DH"""
1951,"""NL""",3.957773,"""No DH"""
2007,"""NL""",4.436638,"""No DH"""
1971,"""AL""",3.467517,"""No DH"""


In [113]:
league_era.write_csv('./data/league_era.csv')

In [114]:
(batting_data := pl.read_csv("./data/lahman_1871-2024_csv/Batting.csv")).head()

playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",11,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2006,1,"""CHN""","""NL""",45,null,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,null
"""aardsda01""",2007,1,"""CHA""","""AL""",25,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2008,1,"""BOS""","""AL""",47,null,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,null
"""aardsda01""",2009,1,"""SEA""","""AL""",73,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null


In [115]:
league_slg = (batting_data
    .filter(pl.col("yearID") >= 1946)
    #Calculate Total Bases
    .with_columns(
        (pl.col("H") - pl.col("2B") - pl.col("3B") - pl.col("HR")).alias("1B"),
        (
            (pl.col("H") - pl.col("2B") - pl.col("3B") - pl.col("HR"))*1 +
            pl.col("2B")*2 +
            pl.col("3B")*3 +
            pl.col("HR")*4
        ).alias("TB")
    )
    .group_by('yearID','lgID')
    .agg(
        tot_bases_league = pl.col('TB').sum(),
        tot_at_bats_league = pl.col('AB').sum(),
        )
     .with_columns( league_wide_SLG = (pl.col('tot_bases_league')/pl.col('tot_at_bats_league')))
     .with_columns( DH_status = pl.when(  ((pl.col('lgID') == 'AL') & (pl.col('yearID') >= 1973)) |
                                         ((pl.col('lgID') == 'NL') & (pl.col('yearID') >= 2022))
                                      )
                   .then(pl.lit('DH'))
                   .otherwise(pl.lit('No DH'))
                 )
     .sort('yearID', descending=True)
)
league_slg.select(pl.col('yearID','lgID','league_wide_SLG','DH_status')).head(10)



yearID,lgID,league_wide_SLG,DH_status
i64,str,f64,str
2024,"""NL""",0.403911,"""DH"""
2024,"""AL""",0.394495,"""DH"""
2023,"""AL""",0.411824,"""DH"""
2023,"""NL""",0.416498,"""DH"""
2022,"""AL""",0.391806,"""DH"""
2022,"""NL""",0.397923,"""DH"""
2021,"""AL""",0.414682,"""DH"""
2021,"""NL""",0.406828,"""No DH"""
2020,"""AL""",0.414228,"""DH"""


In [116]:
league_slg.write_csv('./data/league_slg.csv')